# Solving the *Aristotle38* puzzle using backtracking

Inspired from this [post](https://www.codesdope.com/blog/article/solving-sudoku-with-backtracking-c-java-and-python/) solving Sudoku.  
Check also this cool introduction to backtracking from [Brilliant](https://brilliant.org/wiki/recursive-backtracking/).

In [1]:
import time
import numpy as np

from IPython.display import clear_output

In [2]:
np.random.seed(8)

*Aristotle38* problem.  
19 pieces, each holding a number from 1 to 19, on a board like the following:

<img src="board.png" width=200>

Goal: all visible alignments (in all directions) must sum to 38.

We define a `Board` class, holding the board, a pretty-print function, and the logic to solve the puzzle, namely:  
- a function to get the next empty cell / check if all cells are filled
- a function to test if a board setup is valid
- a recursive backtracking function

In [3]:
class Board:
    
    size = 19
    target = 38
    empty = 0
    
    def __init__(self, fillOrder, testOrder, wait=0):
        """
        create an empty board. 
        """
        _okfillorder = ('rows', 'exterior', 'interior', 'random')
        if fillOrder not in _okfillorder:
            raise ValueError('fillOrder not in {}'.format(_okfillorder))
        if fillOrder == 'rows':
            self._fillOrder = range(self.size)
        elif fillOrder == 'interior':
            self._fillOrder = (9,4,5,10,14,13,8,0,1,2,6,11,15,18,17,16,12,7,3)
        elif fillOrder == 'exterior':
            self._fillOrder = (0,1,2,6,11,15,18,17,16,12,7,3,4,5,10,14,13,8,9)            
        elif fillOrder == 'random':
            self._fillOrder = np.random.choice(range(Board.size),Board.size,replace=False)
            
        _oktestorder = ('linear', 'random')
        if testOrder not in _oktestorder:
            raise ValueError('testOrder not in {}'.format(_oktestorder))
        if testOrder == 'linear':
            self._testOrder = range(1,Board.size+1)
        elif testOrder == 'random':
            self._testOrder = np.random.choice(range(1,Board.size+1),Board.size,replace=False)

        self.board = [Board.empty] * Board.size
        self._wait = wait
        
    def __str__(self):
        """
        pretty-print the board.
        """
        _str = ''
        for i in range(5):       
            if 0 == i:
                _s = ('  ','  ',self.board[0],'  ',self.board[1],'  ',self.board[2],'  ','  ')
            elif 1 == i:
                _s = ('  ',self.board[3],'  ',self.board[4],'  ',self.board[5],'  ',self.board[6],'  ')
            elif 2 == i:
                _s = (self.board[7],'  ',self.board[8],'  ',self.board[9],'  ',self.board[10],'  ',self.board[11])
            elif 3 == i:
                _s = ('  ',self.board[12],'  ',self.board[13],'  ',self.board[14],'  ',self.board[15],'  ')
            elif 4 == i:
                _s = ('  ','  ',self.board[16],'  ',self.board[17],'  ',self.board[18],'  ','  ')
            _str += '{}{}{}{}{}{}{}{}{}\n'.format(*_s)
        _str += '\n'                       
        return _str
    
    def _nextEmpty(self):
        """
        return the next empty cell, depending on `self._fillOrder`,
        or -1 if the board is completed.
        """
        for i in self._fillOrder:
            if Board.empty == self.board[i]:
                return i
        return -1    
    
    def _isValid(self):
        """
        tell whether the current board is valid.
        """
        # there must be no doublon
        count = [0] * (Board.size + 1)
        for n in self.board:
            count[n] += 1
            if 0 < n and 1 < count[n]:
                return False
        
        # all alignments must sum to .target
        check_list = ((0,1,2), # rows
                      (3,4,5,6),
                      (7,8,9,10,11),
                      (12,13,14,15),
                      (16,17,18),
                      (7,3,0), # left rotation rows
                      (12,8,4,1),
                      (16,13,9,5,2),
                      (17,14,10,6),
                      (18,15,11),
                      (2,6,11), # right rotation rows
                      (1,5,10,15),
                      (0,4,9,14,18),
                      (3,8,13,17),
                      (7,12,16))        
        for c in check_list:
            if Board.target < sum(self.board[i] for i in c): # target is overshot
                return False
            if (sum(self.board[i] == Board.empty for i in c) == 0) and (Board.target != sum(self.board[i] for i in c)): # row is full and do not match target
                return False
        return True   

    def solve(self):
        """
        recursive backtracking solver.
        """
        idx  = self._nextEmpty()
        
        # base case    
        if -1 == idx:
            print('found solution:')
            print(self)
            return True

        # else, choose a number between 1 and 19 (depending on ._testOrder)

        for n in self._testOrder:

            # assign n to board[idx]
            self.board[idx] = n
            
            if self._wait > 0: # live printer: from [here](https://stackoverflow.com/questions/465348/how-can-i-print-over-the-current-line-in-a-command-line-application)
                time.sleep(self._wait)
                print(self)
                clear_output(wait=True)
               
            # check if this works
            if self._isValid() and self.solve():
                return True

            # otherwise, backtrack
            else:
                self.board[idx] = Board.empty

        return False

In [ ]:
b = Board('exterior', 'random', 0.001)
b.solve()

    15  7  16    
  13  5  6  4  
10  0  0  0  18
  9  0  0  3  
    19  2  17    


